In [1]:
import sys
sys.path.append('../implementation')
import numpy as np
import pandas as pd
import ast
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
import time
from util import flatten_list
from ottley_hidden_markov_model import HMM
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading the STL Crimes underlying data and user interaction data
underlying_data = pd.read_csv('../data/stl_crimes/dots.csv')
underlying_data.set_index('id', drop=True, inplace=True)
output_file_path = '../output/stl/stl_map_results_hmm.pkl'

interaction_data = pd.read_csv('../data/stl_crimes/stl_combined_interactions.csv')
interaction_data['interaction_session'] = interaction_data.apply(lambda row: ast.literal_eval(row.interaction_session), axis=1)
interaction_data['interaction_type_session'] = interaction_data.apply(lambda row: ast.literal_eval(row.interaction_type_session), axis=1)
ks = [1, 5, 10, 20, 50, 100]

In [ ]:
interaction_index = 2

hmm = HMM(underlying_data, [['x', 'y']], ['type'], 1000)
predicted = pd.DataFrame()
rank_predicted = []
for i in tqdm(range(len(interaction_data.iloc[interaction_index].interaction_session))):
    interaction = interaction_data.iloc[interaction_index].interaction_session[i]
    hmm.update(interaction)
    
    if i < len(interaction_data.iloc[interaction_index].interaction_session) - 1:
        probability_of_next_point = hmm.predict()
        next_point = interaction_data.iloc[interaction_index].interaction_session[i+1]
        predicted_next_dict = {}
        for k in ks:
            predicted_next_dict[k] = (next_point in probability_of_next_point.nlargest(k).index.values)
        sorted_prob = probability_of_next_point.sort_values(ascending=False)
        rank, = np.where(sorted_prob.index.values == next_point)
        rank_predicted.append(rank[0])
        predicted = predicted.append(predicted_next_dict, ignore_index=True)
print(rank_predicted)
    

In [3]:
# Not necessary to run if we already have results file for HMM
# Running HMM through all user interaction sessions and saving results in file
hmm_results = pd.DataFrame()

for participant_index, row in interaction_data.iterrows():
    print(f'Processing user {row.user} task {row.task}')
    results = {'participant_id': row.user, 'task': row.task}
    hmm = HMM(underlying_data, [['x', 'y']], ['type'], 1000)
    predicted = pd.DataFrame()
    rank_predicted = []
    for i in tqdm(range(len(interaction_data.iloc[participant_index].interaction_session))):
        interaction = interaction_data.iloc[participant_index].interaction_session[i]
        hmm.update(interaction)

        if i < len(interaction_data.iloc[participant_index].interaction_session) - 1:
            probability_of_next_point = hmm.predict()
            next_point = interaction_data.iloc[participant_index].interaction_session[i+1]
            predicted_next_dict = {}
            for k in ks:
                predicted_next_dict[k] = (next_point in probability_of_next_point.nlargest(k).index.values)
            predicted = predicted.append(predicted_next_dict, ignore_index=True)
            sorted_prob = probability_of_next_point.sort_values(ascending=False)
            rank, = np.where(sorted_prob.index.values == next_point)
            rank_predicted.append(rank[0] + 1)
            
    ncp = predicted.sum()/len(predicted)
    for col in ncp.index:
        results[f'ncp-{col}'] = ncp[col]
        
    bias = hmm.get_attribute_bias()
    for col in bias.columns:
        results[f'bias-{col}'] = bias[col].to_numpy()
    
    results['bias-mixed'] = results['bias-bias_x___y'] * results['bias-bias_type']
    results['rank'] = rank_predicted
    hmm_results = hmm_results.append(results, ignore_index=True)
    
hmm_results.to_pickle(output_file_path)

  0%|          | 0/54 [00:00<?, ?it/s]

Processing user 15 task geo-based


  2%|▏         | 1/46 [00:00<00:05,  8.09it/s]

Processing user 14 task geo-based


  2%|▏         | 1/47 [00:00<00:05,  8.62it/s]

...

  0%|          | 0/11 [00:00<?, ?it/s]

Processing user 19 task type-based


 17%|█▋        | 2/12 [00:00<00:00, 10.40it/s]

Processing user 18 task type-based


100%|██████████| 12/12 [00:01<00:00, 10.39it/s]
